In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import cv2
from glob import glob
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Defining Path of Dataset
BASE_PATH = r"C:\DeefFake Project\431 MB"
TRAIN_REAL_DIR = os.path.join(BASE_PATH,"Train","training_real")
TRAIN_FAKE_DIR = os.path.join(BASE_PATH, "Train","training_fake")
TEST_REAL_DIR = os.path.join(BASE_PATH, "Test", "Real")
TEST_FAKE_DIR = os.path.join(BASE_PATH, "Test", "Fake")

In [3]:
# Loading Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
def preprocess_images(input_real_dir, input_fake_dir, output_dir):
    
    processed_real_dir = os.path.join(output_dir, "real")
    processed_fake_dir = os.path.join(output_dir, "fake")
    if not os.path.exists(processed_real_dir):
        os.makedirs(processed_real_dir)
    if not os.path.exists(processed_fake_dir):
        os.makedirs(processed_fake_dir)

    # Process real images
    real_image_paths = glob(os.path.join(input_real_dir, '*.jpg'))
    for img_path in tqdm(real_image_paths, desc=f"Cropping real faces from {input_real_dir}"):
        img = cv2.imread(img_path)
        if img is None: continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_img = img[y:y+h, x:x+w]
            output_path = os.path.join(processed_real_dir, os.path.basename(img_path))
            cv2.imwrite(output_path, face_img)
    
    # Process fake images
    fake_image_paths = glob(os.path.join(input_fake_dir, '*.jpg'))
    for img_path in tqdm(fake_image_paths, desc=f"Cropping fake faces from {input_fake_dir}"):
        img = cv2.imread(img_path)
        if img is None: continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_img = img[y:y+h, x:x+w]
            output_path = os.path.join(processed_fake_dir, os.path.basename(img_path))
            cv2.imwrite(output_path, face_img)


In [5]:
def preprocess_images(input_real_dir, input_fake_dir, output_dir):
    processed_real_dir = os.path.join(output_dir, "real")
    processed_fake_dir = os.path.join(output_dir, "fake")
    os.makedirs(processed_real_dir, exist_ok=True)
    os.makedirs(processed_fake_dir, exist_ok=True)

    saved_real = 0
    saved_fake = 0

    # Process real images
    real_image_paths = glob(os.path.join(input_real_dir, '*.jpg'))
    for img_path in tqdm(real_image_paths, desc=f"Processing real images"):
        img = cv2.imread(img_path)
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_img = img[y:y+h, x:x+w]
        else:
            face_img = img  # fallback: use full image

        output_path = os.path.join(processed_real_dir, os.path.basename(img_path))
        cv2.imwrite(output_path, face_img)
        saved_real += 1

    # Process fake images
    fake_image_paths = glob(os.path.join(input_fake_dir, '*.jpg'))
    for img_path in tqdm(fake_image_paths, desc=f"Processing fake images"):
        img = cv2.imread(img_path)
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_img = img[y:y+h, x:x+w]
        else:
            face_img = img  # fallback: use full image

        output_path = os.path.join(processed_fake_dir, os.path.basename(img_path))
        cv2.imwrite(output_path, face_img)
        saved_fake += 1

    # Logging
    print(f"\n✅ Saved {saved_real} real images to {processed_real_dir}")
    print(f"✅ Saved {saved_fake} fake images to {processed_fake_dir}")


In [6]:
# Creating directories for preprocessed data
PROCESSED_DATA_DIR = 'data/processed1'
PROCESSED_TRAIN_DIR = os.path.join(PROCESSED_DATA_DIR, "train")
PROCESSED_TEST_DIR = os.path.join(PROCESSED_DATA_DIR, "test")

print("Preprocessing training images...")
preprocess_images(TRAIN_REAL_DIR, TRAIN_FAKE_DIR, PROCESSED_TRAIN_DIR)
print("\nPreprocessing testing images...")
preprocess_images(TEST_REAL_DIR, TEST_FAKE_DIR, PROCESSED_TEST_DIR)


Preprocessing training images...


Processing real images:   0%|          | 0/16081 [00:00<?, ?it/s]

Processing fake images:   0%|          | 0/15960 [00:00<?, ?it/s]


✅ Saved 16081 real images to data/processed1\train\real
✅ Saved 15960 fake images to data/processed1\train\fake

Preprocessing testing images...


Processing real images:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing fake images:   0%|          | 0/5000 [00:00<?, ?it/s]


✅ Saved 5000 real images to data/processed1\test\real
✅ Saved 5000 fake images to data/processed1\test\fake


In [7]:
class DeepFakeDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.real_files = glob(os.path.join(real_dir, '*.jpg'))
        self.fake_files = glob(os.path.join(fake_dir, '*.jpg'))
        self.files = self.real_files + self.fake_files
        self.labels = [0] * len(self.real_files) + [1] * len(self.fake_files)
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = self.files[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        return image, label


In [8]:
# Defining image transformations
image_transforms = transforms.Compose([
    transforms.Resize((299, 299)),  # XceptionNet requires 299x299
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [9]:
# Creating the datasets
train_dataset = DeepFakeDataset(os.path.join(PROCESSED_TRAIN_DIR, "real"), os.path.join(PROCESSED_TRAIN_DIR, "fake"), transform=image_transforms)
test_dataset = DeepFakeDataset(os.path.join(PROCESSED_TEST_DIR, "real"), os.path.join(PROCESSED_TEST_DIR, "fake"), transform=image_transforms)


In [10]:
# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")


Number of training samples: 32041
Number of testing samples: 10000


In [11]:
import timm
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('xception', pretrained=True)

# Modifying the final layer for binary classification
num_ftrs = model.get_classifier().in_features
model.fc = nn.Linear(num_ftrs, 1)

model = model.to(device)

print("Model loaded and configured successfully.")


c:\VS Code\Jupyter Notebook\src\virenv\Lib\site-packages\timm\models\_factory.py:138: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Model loaded and configured successfully.


In [12]:
# loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [13]:
# Training loop
num_epochs = 5
best_val_acc = 0.0
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.float().to(device).unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f"Training Loss: {epoch_loss:.4f}")


Epoch 1/5


Training:   0%|          | 0/1002 [00:00<?, ?it/s]

Training Loss: 0.1828
Epoch 2/5


Training:   0%|          | 0/1002 [00:00<?, ?it/s]

Training Loss: 0.1007
Epoch 3/5


...

Training Loss: 0.0814
Epoch 4/5


Training:   0%|          | 0/1002 [00:00<?, ?it/s]

Training Loss: 0.0753
Epoch 5/5


Training:   0%|          | 0/1002 [00:00<?, ?it/s]

Training Loss: 0.0683


In [14]:
# Evaluating model
model.eval()
test_preds = []
test_labels_list = []
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.float().to(device).unsqueeze(1)
        outputs = model(images)
        preds = torch.sigmoid(outputs)
        test_preds.extend(preds.cpu().numpy())
        test_labels_list.extend(labels.cpu().numpy())


Testing:   0%|          | 0/313 [00:00<?, ?it/s]

In [15]:
# Calculating test metrics
test_preds_binary = [1 if p > 0.5 else 0 for p in test_preds]
test_acc = accuracy_score(test_labels_list, test_preds_binary)
test_precision = precision_score(test_labels_list, test_preds_binary)
test_recall = recall_score(test_labels_list, test_preds_binary)
test_f1 = f1_score(test_labels_list, test_preds_binary)

print("\n--- Test Set Performance ---")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-Score: {test_f1:.4f}")

def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return "Error: Could not read image.", 0.0
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        return "No face detected.", 0.0
    
    (x, y, w, h) = faces[0]
    face_img = img[y:y+h, x:x+w]
    
    # Converting back to PIL Image and apply transformations
    face_img_pil = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
    input_tensor = image_transforms(face_img_pil).unsqueeze(0).to(device)

    # Getting prediction
    with torch.no_grad():
        output = model(input_tensor)
        prediction = torch.sigmoid(output).item()

    if prediction > 0.5:
        return "Prediction: Fake", prediction
    else:
        return "Prediction: Real", prediction



--- Test Set Performance ---
Test Accuracy: 0.9902
Test Precision: 0.9906
Test Recall: 0.9898
Test F1-Score: 0.9902


In [16]:
model.eval()

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [17]:
# Save the model
torch.save(model.state_dict(), "deepfake_xception_model.pth")
print("✅ Model saved successfully as 'deepfake_xception_model.pth'")

✅ Model saved successfully as 'deepfake_xception_model.pth'


In [18]:
torch.save(model.state_dict(), "deepfake_model1.pt")
print("✅ Model weights saved as 'deepfake_model1.pt'")

✅ Model weights saved as 'deepfake_model1.pt'


In [19]:
#To Load the model
import timm
import torch.nn as nn

# Recreate the model architecture
model = timm.create_model('xception', pretrained=False)
num_ftrs = model.get_classifier().in_features
model.fc = nn.Linear(num_ftrs, 1)

# Load the saved weights
model.load_state_dict(torch.load("deepfake_xception_model.pth"))
model = model.to(device)
model.eval()

print("✅ Model loaded and ready for inference.")

c:\VS Code\Jupyter Notebook\src\virenv\Lib\site-packages\timm\models\_factory.py:138: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


✅ Model loaded and ready for inference.


In [46]:
result, confidence = predict_image(r"C:\Users\T8664\Downloads\Real time images\20250117_222808.jpg")
print(result)
print(f"Confidence: {confidence:.4f}")

Prediction: Real
Confidence: 0.3762


In [1]:
import torch
from torchvision.models import resnet18

model = resnet18(pretrained=True)
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

torch.save(quantized_model.state_dict(), "quantized_model.pth")


c:\VS Code\Jupyter Notebook\src\virenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\VS Code\Jupyter Notebook\src\virenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\T8664/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:30<00:00, 1.56MB/s]
C:\Users\T8664\AppData\Local\Temp\ipykernel_17912\719472180.py:5: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)


In [2]:

def split_file(file_path, chunk_size_mb=25):
    chunk_size = chunk_size_mb * 1024 * 1024
    base_name = os.path.basename(file_path)
    dir_name = os.path.dirname(file_path)

    with open(file_path, 'rb') as f:
        data = f.read()

    chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

    for idx, chunk in enumerate(chunks):
        chunk_filename = os.path.join(dir_name, f"{base_name}.part{idx+1}")
        with open(chunk_filename, 'wb') as chunk_file:
            chunk_file.write(chunk)

    print(f"Split {base_name} into {len(chunks)} parts under {chunk_size_mb}MB each.")

In [3]:
import torch
import os
import zipfile

# === CONFIGURATION ===
model_path = "C:/VS Code/Jupyter Notebook/quantized_model.pth"
compressed_path = "C:/VS Code/Jupyter Notebook/quantized_model_compressed.zip"

# Ensure the directory exists
os.makedirs(os.path.dirname(compressed_path), exist_ok=True)

# === Compress the model file ===
with zipfile.ZipFile(compressed_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(model_path, arcname=os.path.basename(model_path))

# === Check compressed size ===
compressed_size_mb = os.path.getsize(compressed_path) / (1024 * 1024)
print(f"Compressed model size: {compressed_size_mb:.2f} MB")

if compressed_size_mb <= 25:
    print("✅ Compression successful: File is under 25MB.")
else:
    print("⚠️ Compression not sufficient: File still exceeds 25MB.")

Compressed model size: 39.98 MB
⚠️ Compression not sufficient: File still exceeds 25MB.
